In [1]:
import csv
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from datetime import datetime, timedelta
from tqdm import tqdm
import json
kma_data = pd.read_csv('weather_20180101_20240523_2.csv')
naver_data = pd.read_csv('new_data.csv')

In [2]:
# tm 날짜
# ta 기온
# rn 강수량
# ws 풍속
# wd 풍향
# hm 습도
# pa 현지기압
# ss 일조
# dc10Tca 전운량
# dmstMtphNo 현상번호 - 눈 비인지 알수있음

In [3]:
kma_data = kma_data[["tm","ta","rn","ws","wd","hm","pa","ss","dc10Tca","dmstMtphNo"]]

In [4]:
# 필요한 컬럼만 정제해서 저장
kma_data.to_csv('kma_data.csv', index=False, encoding='utf-8-sig')

In [5]:
kma_data.isna().sum()

tm                0
ta                1
rn            49739
ws               79
wd               79
hm                0
pa               10
ss            25340
dc10Tca        1118
dmstMtphNo    39283
dtype: int64

In [6]:
naver_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1583 entries, 0 to 1582
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   aplYmd   1582 non-null   float64
 1   lareaNm  1582 non-null   object 
 2   mareaNm  1582 non-null   object 
 3   wetrCd   1582 non-null   float64
 4   wetrTxt  1582 non-null   object 
dtypes: float64(2), object(3)
memory usage: 62.0+ KB


In [7]:
print(naver_data[naver_data['aplYmd'].isna()])
print(naver_data[naver_data['aplYmd'] == float('inf')]) 
# 지금 1581 행이 다 결측값으로 들어가있음 왜지??

      aplYmd lareaNm mareaNm  wetrCd wetrTxt
1582     NaN     NaN     NaN     NaN     NaN
Empty DataFrame
Columns: [aplYmd, lareaNm, mareaNm, wetrCd, wetrTxt]
Index: []


In [8]:
# 마지막 행 삭제
naver_data.drop(naver_data.tail(1).index, inplace=True)

In [9]:
print(naver_data[naver_data['aplYmd'].isna()])
print(naver_data[naver_data['aplYmd'] == float('inf')]) 

Empty DataFrame
Columns: [aplYmd, lareaNm, mareaNm, wetrCd, wetrTxt]
Index: []
Empty DataFrame
Columns: [aplYmd, lareaNm, mareaNm, wetrCd, wetrTxt]
Index: []


In [10]:
# 왜 float으로 되어있는지는 모르지만 int형으로 변경
naver_data["aplYmd"] = naver_data['aplYmd'].astype(int)

In [11]:
# join 하기위해서 새로운 컬럼 생성
kma_data["new_tm"] = pd.to_datetime(kma_data['tm']).dt.strftime('%Y%m%d')

naver_data의 aplYmd값이 20191229에서 20240524까지 있으므로 kma_data도 맞추어서 정리

In [12]:
# datetime 형식으로 변환
kma_data['new_tm'] = pd.to_datetime(kma_data['new_tm'])

In [13]:
filter_kma_data = kma_data[(kma_data['new_tm'] >= '2019-12-29') & (kma_data['new_tm'] <= '2024-05-24')]

In [14]:
# kma_data가 20240523까지밖에 없으므로 naver_data 20240524 로우 삭제
naver_data.drop(naver_data.tail(1).index, inplace=True)

In [15]:
# join 하기 위해서 다시 int로
filter_kma_data['new_tm'] = pd.to_datetime(filter_kma_data['tm']).dt.strftime('%Y%m%d')

/tmp/ipykernel_2165/1065656092.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_kma_data['new_tm'] = pd.to_datetime(filter_kma_data['tm']).dt.strftime('%Y%m%d')


In [16]:
filter_kma_data.head()

,tm,ta,rn,ws,wd,hm,pa,ss,dc10Tca,dmstMtphNo,new_tm
17304,2019-12-29 00:00,1.4,NaN,1.8,50.0,57,1018.2,NaN,3.0,NaN,20191229
17305,2019-12-29 01:00,1.2,NaN,1.8,70.0,54,1017.9,NaN,6.0,NaN,20191229
17306,2019-12-29 02:00,1.3,NaN,2.4,70.0,53,1017.9,NaN,4.0,NaN,20191229
17307,2019-12-29 03:00,1.4,NaN,1.8,70.0,52,1017.9,NaN,5.0,NaN,20191229
17308,2019-12-29 04:00,1.4,NaN,2.6,70.0,51,1017.1,NaN,5.0,NaN,20191229


In [17]:
naver_data.head()

,aplYmd,lareaNm,mareaNm,wetrCd,wetrTxt
0,20191229,서울특별시,종로구,9.0,흐림/비
1,20191230,서울특별시,종로구,12.0,구름많음/눈/비/안개
2,20191231,서울특별시,종로구,12.0,맑음/눈
3,20200101,서울특별시,종로구,12.0,흐림/눈/안개
4,20200102,서울특별시,종로구,17.0,구름많음/안개


In [18]:
# join 하기위해서 colum명 변경 어차피 저거 임시로 만든컬럼이라 ㄴ 상관
filter_kma_data.rename(columns={'new_tm': 'aplYmd'}, inplace=True)

/tmp/ipykernel_2165/3236204979.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_kma_data.rename(columns={'new_tm': 'aplYmd'}, inplace=True)


In [19]:
# type이 object 여서 int로 바꾸기
filter_kma_data["aplYmd"] = filter_kma_data['aplYmd'].astype(int)

/tmp/ipykernel_2165/677594474.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_kma_data["aplYmd"] = filter_kma_data['aplYmd'].astype(int)


In [20]:
total_df = pd.merge(filter_kma_data, naver_data, on='aplYmd', how='inner')

In [21]:
total_df

,tm,ta,rn,ws,wd,hm,pa,ss,dc10Tca,dmstMtphNo,aplYmd,lareaNm,mareaNm,wetrCd,wetrTxt
0,2019-12-29 00:00,1.4,NaN,1.8,50.0,57,1018.2,NaN,3.0,NaN,20191229,서울특별시,종로구,9.0,흐림/비
1,2019-12-29 01:00,1.2,NaN,1.8,70.0,54,1017.9,NaN,6.0,NaN,20191229,서울특별시,종로구,9.0,흐림/비
2,2019-12-29 02:00,1.3,NaN,2.4,70.0,53,1017.9,NaN,4.0,NaN,20191229,서울특별시,종로구,9.0,흐림/비
3,2019-12-29 03:00,1.4,NaN,1.8,70.0,52,1017.9,NaN,5.0,NaN,20191229,서울특별시,종로구,9.0,흐림/비
4,2019-12-29 04:00,1.4,NaN,2.6,70.0,51,1017.1,NaN,5.0,NaN,20191229,서울특별시,종로구,9.0,흐림/비
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37771,2024-05-23 19:00,22.5,NaN,3.5,270.0,55,999.5,1.0,0.0,NaN,20240523,서울특별시,종로구,17.0,맑음/안개
37772,2024-05-23 20:00,20.5,NaN,3.0,290.0,68,1000.3,0.0,0.0,NaN,20240523,서울특별시,종로구,17.0,맑음/안개
37773,2024-05-23 21:00,18.6,NaN,2.8,270.0,73,1001.0,NaN,0.0,NaN,20240523,서울특별시,종로구,17.0,맑음/안개
37774,2024-05-23 22:00,17.7,NaN,2.9,290.0,79,1001.6,NaN,0.0,NaN,20240523,서울특별시,종로구,17.0,맑음/안개


In [22]:
total_df.to_csv('2020_now_merge_df.csv', index=False, encoding='utf-8-sig')